# Kelly, Bet sizing

In [2]:
import numpy as np
import pandas as pd

In [29]:
from scipy import optimize
from scipy.optimize import minimize_scalar, newton, minimize, fsolve
from scipy.stats import pearson3

from scipy.integrate import quad
from scipy.stats import norm

## Discrete

In [4]:
def optimal_kelly(odds, win_probability):
    edge = (1+odds)*(win_probability)-1
    optimal_fraction = edge/odds
    return optimal_fraction

In [5]:
odds = 1
p = 0.53

In [6]:
optimal_kelly(odds, p)

0.06000000000000005

## Continious

#### Single Asset

Assuming that the probability distribution of returns is Gaussian

$$f = \frac{m}{s^2}$$

In [7]:
# from "2019-03-11" to "2020-03-10"
prices_df = pd.read_csv("sp50_1year.csv", index_col=0)

In [8]:
# Micron Technology
prices_df['MU']

Date
2019-03-11    39.029999
2019-03-12    39.250000
2019-03-13    38.830002
2019-03-14    38.410000
2019-03-15    39.540001
                ...    
2020-03-04    55.290001
2020-03-05    53.720001
2020-03-06    51.470001
2020-03-09    45.970001
2020-03-10    47.860001
Name: MU, Length: 253, dtype: float64

In [9]:
daily_log_returns = np.log(prices_df['MU']/prices_df['MU'].shift(1))
daily_log_returns.fillna(0, inplace=True)

In [10]:
def optimal_kelly_leverage(excess_return_series, risk_free, annualize_factor):
    """
    In this method, we assume that our distribution of excess return is Gaussian.
    We can expect maximized growth rate of our asset if we keep kelly leverage.
    """
    annualized_log_returns = excess_return_series*annualize_factor
    annualized_std = np.std(excess_return_series)*np.sqrt(annualize_factor)
    
    mean_excess_return = np.mean(annualized_log_returns) - risk_free
    
    kelly_leverage = mean_excess_return/annualized_std**2
    maximum_growth_rate = risk_free + kelly_leverage*mean_excess_return - (annualized_std*kelly_leverage)**2/2
    
    return kelly_leverage, maximum_growth_rate

In [11]:
optimal_kelly_leverage(daily_log_returns, risk_free=0.01, annualize_factor=256)

(0.9623939353487783, 0.10449164735019847)

#### Montecarlo optimal leverage


In [12]:
def g(f, R):
    return np.sum(np.log(1+f*R), axis=0)/R.shape[0]

def monte_carlo_optimal_leverage(return_series, random_state, size):
    # Return MLEs for shape (if applicable), location, and scale parameters from data.
    skew_, loc_, scale_ = pearson3.fit(return_series)
    mean, var, skew, kurt = pearson3.stats(skew_, loc_, scale_, moments='mvks')
    
    return_samples = pearson3.rvs(skew_, loc_, scale_, size=size, random_state=random_state)
    
    minusGsim = lambda f : -g(f, return_samples)
    res = minimize(minusGsim, 0, method='Nelder-Mead')
    optimalF=res.x
    
    print('Optimal leverage=%f optimal growth rate=%f' % (optimalF, -res.fun))
    
    return optimalF, -res.fun

In [14]:
monte_carlo_optimal_leverage(daily_log_returns*256, random_state=0, size=10000)

Optimal leverage=0.004563 optimal growth rate=0.000532


(array([0.0045625]), 0.0005321404727280112)

#### multiple assets, Optimal Allocation

In [15]:
from numpy.linalg import inv

$$F^* = C^{-1}M$$

In [16]:
tickers = list(prices_df)[:5]
tickers

['ADM', 'AGN', 'AIV', 'ALL', 'ALLE']

In [17]:
target = prices_df[tickers]
log_returns = np.log(target/target.shift(1)).dropna()

In [18]:
log_returns

,ADM,AGN,AIV,ALL,ALLE
Date,,,,,
2019-03-12,0.001633,0.007609,0.009189,-0.000425,0.002593
2019-03-13,0.008355,0.018174,0.000596,-0.005543,0.009302
2019-03-14,0.001847,-0.004462,0.008903,0.003202,-0.006736
2019-03-15,-0.003929,0.009633,0.000197,0.010493,-0.004064
2019-03-18,-0.016344,-0.002715,-0.019688,0.001896,0.005527
...,...,...,...,...,...
2020-03-04,0.033840,0.020626,0.034917,0.047481,0.041124
2020-03-05,-0.026771,-0.015688,-0.017306,-0.054281,-0.013765
2020-03-06,-0.018166,-0.004885,-0.011974,-0.012245,-0.023511


In [19]:
risk_free = 0.01

In [20]:
def optimal_kelly_allocation(returns_matrix, risk_free, annualize_factor):
    """
    """
    excess_returns = returns_matrix - risk_free/annualize_factor
    M = excess_returns.mean() * annualize_factor
    C = excess_returns.cov() * annualize_factor
    
    # Kelly
    F = np.dot(inv(C), M)
    
    # Annualized compounded growth rate
    g = risk_free + F.T.dot(C).dot(F)/2
    return F, g

In [21]:
F, g = optimal_kelly_allocation(log_returns, risk_free=0.01, annualize_factor=256)
F, g

(array([-5.96507684,  2.63869778, -2.38328886,  1.05329052,  8.21475191]),
 1.961056422047869)

#### The kelly Criterion and the Sotck market 

resource: [Betting with the Kelly criterion](https://quantdare.com/kelly-criterion/)

In [22]:
def norm_dev_integral(f, m, st):
    val,er = quad(lambda s: (s/(1+f*s))*norm.pdf(s,m,st),m-3*st,m+3*st)
    return val

In [23]:
# Reference values from Eduard Thorp's article
m = .058
s = .216

x0 = newton(norm_dev_integral, 1.0, args=(m,s))
print('Optimal Kelly fraction: {:.4f}'.format(x0))


Optimal Kelly fraction: 1.1974


I changed above a little to implement the paper as same as possible. I'm not sure this is correct

> paper: The Kelly Criterion and the Stock Market by Louis M.Rotando and Edward O. Thorp

In [27]:
def get_adjusted_sigma(mu, sigma, h):
    def func(alpha, mu=mu, sigma=sigma, h=h):
        value, error = quad(lambda s: s**2*(norm.pdf(s, mu, sigma)+h), mu-3*sigma, mu+3*sigma)
        return value-mu**2-sigma**2
    
    # use std as initial value
    answer = optimize.fsolve(func, sigma)
    return answer[0]
    
def quasi_normal_dev_integral(fraction, mu, sigma):
    
    # base_line = (B-A)
    base_line= (mu+3*sigma) - (mu-3*sigma)

    # h = area/base_line
    h = (1-(norm(0, 1).cdf(3.) - norm(0, 1).cdf(-3.)))/base_line
    
    alpha = get_adjusted_sigma(mu, sigma, h)
    
    pdf_of_return = lambda x: norm.pdf(x, mu, alpha)+h
        
    value, error = quad(lambda s:(s/(1+fraction*s)) * pdf_of_return(s), mu-3*sigma, mu+3*sigma)
    return value

In [30]:
# Reference values from Eduard Thorp's article
m = .058
s = .216
x0 = newton(quasi_normal_dev_integral,1.0,args=(m,s))
print('Optimal Kelly fraction: {:.4f}'.format(x0))


/opt/conda/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


Optimal Kelly fraction: 1.1906
